In [ ]:
import robotic as ry
import numpy as np
import time
import random

ry.params_add({'botsim/verbose': 1., 'physx/motorKp': 10000., 'physx/motorKd': 1000.})
ry.params_add({'botsim/engine': 'physx'}) #makes a big difference!
ry.params_add({'physx/multibody': True}) #makes a big difference!

C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))
C.addFile("obs_around_env.g")
C.computeCollisions()
#print(C.getCollisions())
C.view(False, 'this is your workspace data structure C -- NOT THE SIMULTATION')
bot = ry.BotOp(C, useRealRobot=False)

box = C.frame("box")
box_size = box.getSize()
print(box_size)

qF = [-0.5,0.5,0.68]

def IK(C, target, qHome):
    q0 = C.getJointState()
    komo = ry.KOMO(C, 1, 1, 0, False) #one phase one time slice problem, with 'delta_t=1', order=0
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], q0) #cost: close to 'current state'
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], qHome) #cost: close to qHome
    komo.addObjective([], ry.FS.poseDiff, ['l_gripper', target], ry.OT.eq, [1e2]) #constraint: gripper position
    ret = ry.NLP_Solver(komo.nlp(), verbose=0) .solve()
    return [komo.getPath()[0], ret]


def RRT(qF, rrt_extend_length=0.04):
    C2 = ry.Config()
    C2.addFile("maze_obs_around_actuated_env.g")

    q0 = C2.getJointState()
    print("Start State: ", q0," Final State: ", qF)
    C2.view()

    # Create an RRT path finder
    rrt = ry.PathFinder()
    rrt.setProblem(C2, [q0], [qF])
    
    isFeas = False
    path = None

    i=0
    # Find an RRT path
    while i!=10 and (not isFeas):
        rrt = ry.PathFinder()
        rrt.setProblem(C2, [q0], [qF])
        ret = rrt.solve()
        isFeas = ret.feasible
        path = ret.x
        i+=1
    
    ## Smooth the path (optimization) ##
    print("Path Prev:", len(path))
    #Create a KOMO for post-optimization with acceleration constraints
    num_time_steps = len(path)

    komo2 = ry.KOMO(C2, num_time_steps, 1, 2, True)
    #Add an acceleration constraint objective to the new KOMO object
    komo2.addControlObjective([], 2, 1e0)
    komo2.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq,[1e2])
    #Set the waypoints for the KOMO
    komo2.initWithPath_qOrg(path)

    ret = ry.NLP_Solver() \
        .setProblem(komo2.nlp()) \
        .setOptions(stopTolerance=1e-2, verbose=0) \
        .solve()
    print(ret)

    path = komo2.getPath()
    print("Path Post:", len(path))
    new_path = path

    for i in range(0, len(new_path)):
        C2.setJointState(new_path[i])
        C2.view()
        time.sleep(.1)
    
    return new_path

def addMarker(C, pos, name, parent, size=0.1):
    marker = C.addFrame(name, parent= parent) \
              .setShape(ry.ST.ssBox, size = [0, 0, 0, .001]) \
              .setPosition(pos)
    C.view()
    return marker

def addPoint(C, pos, name, parent, color):
    point = C.addFrame(name, parent= parent) \
              .setShape(ry.ST.sphere, size = [0, 0, 0, .02]) \
              .setPosition(pos) \
              .setColor(color) \
              .setContact(0)
    C.view()
    return point

def addPoint2(C, obj, name, parent, color):
    pos = C.frame(obj).getPosition()
    point = C.addFrame(name, parent= parent) \
              .setShape(ry.ST.sphere, size = [0, 0, 0, .02]) \
              .setPosition(pos) \
              .setColor(color) \
              .setContact(0)
    C.view()
    return point

def calculateContactPoint(middle_point, target_point, size):
    """
    This function is used to find the contact point to move the target object to the designated goal. 
    """
    radius = size[0]*3
    # Calculate the vector from the middle point to the near point
    vector = target_point - middle_point
    # Normalize the vector to make it a unit vector
    normalized_vector = vector / np.linalg.norm(vector)
    # Calculate the point on the opposite side of the circle
    opposite_point = middle_point - normalized_vector * radius
    return opposite_point 

def calculateWayPoint(middle_point, target_point, size):
    """
    This function is used to find the contact point to move the target object to the designated goal. 
    """
    radius = size[0]
    # Calculate the vector from the middle point to the near point
    vector = target_point - middle_point
    # Normalize the vector to make it a unit vector
    normalized_vector = vector / np.linalg.norm(vector)
    # Calculate the point on the opposite side of the circle
    opposite_point = middle_point - normalized_vector * radius
    return opposite_point 

def moveBotAlongPath4(C, bot, path, contact_point="", obstacle_name=""):
    qHome = bot.get_qHome()

    addPoint(C, C.frame("cpoint").getPosition(), "contact"+str(random.randint(0,10000)), "pawn_handle", [1, 1, 0])
    q0, ret = IK(C, "cpoint", qHome)
    bot.moveTo(q0, timeCost=3., overwrite=True)
    
    while(bot.getTimeToEnd() > 0):
        bot.sync(C, 1) #keep the workspace C sync'ed to real/sim, and idle .1 sec

    bot.gripperMove(ry._left, width=.005, speed=.2)
    while not bot.gripperDone(ry._left):
        bot.sync(C, .1)


    q1, ret = IK(C, "cpoint2", qHome)
    bot.moveTo(q1, timeCost=.1, overwrite=True)
    while(bot.getTimeToEnd() > 0):
        bot.sync(C, 1) #keep the workspace C sync'ed to real/sim, and idle .1 sec

    for i in range(len(path)):
        addMarker(C, [*path[i][:2], 0.755], "targetWP", "world", 0.07)
        contact_point = [*calculateContactPoint(box.getPosition(), [*path[i][:2], box.getPosition()[2]], box_size)[:2], 0.755]
        addMarker(C, contact_point, "targetCP", "world", 0.07)
        q_target2, ret = IK(C, "targetCP", qHome)
        print(f"Contact Point {contact_point}")
        bot.moveTo(q_target2, timeCost=.1, overwrite=True)
        while bot.getTimeToEnd()>0:
            bot.sync(C, .1)
        time.sleep(1)

        if i == len(path)-1:
            print(f"WayPoint {qF}")
            way_point = [*calculateWayPoint(box.getPosition(), [*qF[:2], box.getPosition()[2]], box_size)[:2], 0.755]
            addMarker(C, way_point, "targetWP", "world", 0.07)
            q_target2, ret = IK(C, "targetWP", qHome)
            bot.moveTo(q_target2, timeCost=.1, overwrite=True)
            while bot.getTimeToEnd()>0:
                bot.sync(C, .1)
            time.sleep(1)         
            break

        print(f"WayPoint {path[i+1]}")
        
        addPoint(C, box.getPosition(), "contact"+str(random.randint(0,10000)), "world", [1, 0, 0])
        way_point = [*calculateWayPoint(box.getPosition(), [*path[i+1][:2], box.getPosition()[2]], box_size)[:2], 0.755]
        addMarker(C, way_point, "targetWP", "world", 0.07)
        q_target2, ret = IK(C, "targetWP", qHome)
        bot.moveTo(q_target2, timeCost=.1, overwrite=True)
        while bot.getTimeToEnd()>0:
            bot.sync(C, .1)
        time.sleep(1)

def moveToFreeSpace(C, bot, path):
    moveBotAlongPath4(C, bot, path, "contact_point_box", "box")


In [ ]:
qF = [-0.5,0.5,0.68]
addMarker(C,qF,'marker','world')
path = RRT([*qF[:2], 0])

In [ ]:
moveToFreeSpace(C, bot, path)

In [ ]:
for i in C.getFrameNames():
    if "panda" in i or i in ["gripper", "palm", "finger1", "finger2", "l_gripper", "l_palm", "l_finger1", "l_finger2", "cameraTop", "cameraWrist", "bellybutton"]:
        C.delFrame(i)
C.view()

In [ ]:
C.getFrameNames()

In [ ]:
import robotic as ry
import numpy as np
import time
import random

ry.params_add({'botsim/verbose': 1., 'physx/motorKp': 10000., 'physx/motorKd': 1000.})
ry.params_add({'botsim/engine': 'physx'}) #makes a big difference!
ry.params_add({'physx/multibody': True}) #makes a big difference!

C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))
C.addFile("maze_obs_around_env.g")
C.computeCollisions()
#print(C.getCollisions())
C.delFrame("cameraWrist")
C.view(False, 'this is your workspace data structure C -- NOT THE SIMULTATION')